In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown', 
                                            # Update dropdown values using list comphrehension
                                            options=[   {'label': 'All launch sites', 'value': 'all'},
                                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                            value = 'all',
                                            placeholder = "Select a launch site",
                                            style = {'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'left'},
                                            searchable = True
                                            ),
                                html.Br(),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',min=0,max=10000,step=1000, value = [min_payload,max_payload], 
                                                marks = { 0: '0',
                                                        1000: '1000',
                                                        2000: '2000',
                                                        3000: '3000',
                                                        4000: '4000',
                                                        5000: '5000',
                                                        6000: '6000',
                                                        7000: '7000',
                                                        8000: '8000',
                                                        9000: '9000',
                                                        10000: '10000'
                                                        }),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                html.Div(dcc.Graph(id='success-payload-scatter-plot')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( [Output(component_id='success-payload-scatter-chart', component_property='figure'),
                Output(component_id='success-payload-scatter-plot', component_property='figure')],
               [Input(component_id='site-dropdown', component_property='value'),
                Input(component_id='payload-slider', component_property ='value')]
               )
def get_scatter_plot(dropdown, slider):
    
    
    pos_list = slider
    df = spacex_df[spacex_df['Payload Mass (kg)'].between(pos_list[0], pos_list[1])]
    
    if dropdown == 'all':
        plot = px.scatter(df, x='Payload Mass (kg)',  y='class', title='Payload mass vs. launch success for all sites', color = 'Booster Version Category')
        plot1 = px.scatter(df, x='Payload Mass (kg)',  y='Mission Outcome', title='Payload mass vs. launch success for all sites', color = 'Booster Version Category')
    else:
        filtered_df1 = df[df['Launch Site'] == dropdown]
        plot = px.scatter(filtered_df1, x='Payload Mass (kg)',  y='class', title='Payload mass vs. launch success for selected site', color = 'Booster Version Category')
        plot1 = px.scatter(filtered_df1, x='Payload Mass (kg)',  y='Mission Outcome', title='Payload mass vs. launch success for selected site', color = 'Booster Version Category')
    return plot, plot1


@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value')
               )

def get_piechart(value):
    
    if value == 'all':
        #Success Counts for ALL Sites
        success = spacex_df[spacex_df['class']==1]
        piechart = px.pie(success,values='class', names='Launch Site', title='Successful launches for all launch sites')
    else:
        # Success Counts for ONE site
        filtered_df = spacex_df[spacex_df['Launch Site'] == value]
        filtered_df.groupby(['Launch Site','class'])
        piechart = px.pie(filtered_df,values='class', names='Mission Outcome', title='Success launches for launch site '+ str(value))
    return piechart


# Run the app
if __name__ == '__main__':
    app.run_server()



Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Aug/2021 18:28:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 18:28:59] "GET /_dash-component-suites/dash_core_components/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 18:28:59] "GET /_dash-component-suites/dash_core_components/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 18:28:59] "GET /_dash-component-suites/dash_core_components/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 18:28:59] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 18:28:59] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 18:28:59] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 18:29:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 18:29:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 20:11:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/202